In [ ]:
import sys
sys.path.append('src/')
from percentparser import parse_percentage
from plot_functions import single_var_catplot, multi_var_catplot
from stat_tests import check_normality_for_groups, check_variance_homogeneity, kruskal_wallis_test, dunns_test, detailed_dunns_test

import os
import json
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime


import warnings
warnings.filterwarnings("ignore")

sns.set_theme(context='notebook', style='whitegrid')
pd.set_option("display.max_rows", 100)

In [ ]:
start_run = datetime.datetime.now()

# Analysis

In [ ]:
df = pd.read_csv("processed_data/age_name_edu_data.csv.zip", compression='zip')

In [ ]:
len(df)

In [ ]:
assert len(df['custom_id'].unique()) == len(df)

In [ ]:
model = df['model'].unique()[0]

### Response Length and Refusals

In [ ]:
df['response_len'] = df['query_response_raw'].apply(len)
print(f"Median response length: {df['response_len'].median()}")
print(f"Mean response length: {df['response_len'].mean()}")

In [ ]:
refusals = df[df['query_response'].isna()]
print(f"Median refusal response length: {refusals['response_len'].median()}")
print(f"Mean refusal response length: {refusals['response_len'].mean()}")

In [ ]:
refusals

In [ ]:
print(f"Refusal 1: \n{refusals['query_response_raw'].iloc[0]}\n\n\
Refusal 2: \n{refusals['query_response_raw'].iloc[1]}")

In [ ]:
df_valid = df[df['query_response'].notna()]
print(f"Median valid response length: {df_valid['response_len'].median()}")
print(f"Mean valid response length: {df_valid['response_len'].mean()}")

In [ ]:
df['invalid_or_refused'] = df['query_response'].isna().astype(int)
print(f"Invalid responses: {df['invalid_or_refused'].mean()}")

### Response Distribution

In [ ]:
df_valid['query_response'] = pd.to_numeric(df_valid['query_response'], errors='coerce')
print(f"Min hiring probability: {min(df_valid['query_response'])}\nMax hiring probability: {max(df_valid['query_response'])}")

In [ ]:
response_value_counts = df_valid['query_response'].value_counts().to_dict()
pd.DataFrame({'hiring_prob': response_value_counts.keys(),
             'count': response_value_counts.values()}).sort_values(by='hiring_prob')

In [ ]:
plt.bar(response_value_counts.keys(), response_value_counts.values())

## Visualizations

In [ ]:
df_valid.groupby(['age','gender','education']).count()['query_response']

In [ ]:
df_valid.groupby(['age'])['query_response'].agg(['min', 'median', 'max', 'mean', 'std'])

In [ ]:
df_valid.groupby(['gender'])['query_response'].agg(['min', 'median', 'max', 'mean', 'std'])

In [ ]:
df_valid.groupby(['education'])['query_response'].agg(['min', 'median', 'max', 'mean', 'std'])

### Single Variable Plots

In [ ]:
df_valid['age'] = df_valid['age'].astype(str)
plot = single_var_catplot(df_valid, 'age', model)

In [ ]:
gender_stats_diff = df_valid.groupby(['gender'])['query_response'].agg(['mean', 'median'])
gender_stats_diff

In [ ]:
# Calculate the difference between Man and Woman for both mean and median values
gender_mean_gap = gender_stats_diff.unstack()['mean', 'Man'] - gender_stats_diff.unstack()['mean', 'Woman']

print(f"Mean gender gap (male vs female): {gender_mean_gap}")

In [ ]:
single_var_catplot(df_valid, 'gender', model)

In [ ]:
single_var_catplot(df_valid, 'education', model)

### Multi Variable Plots

In [ ]:
categories = ['age', 'gender', 'education']
for var1 in categories:
    for var2 in categories:
        if var1 != var2:
            multi_var_catplot(df_valid, var1, var2, model)

In [ ]:
heatmap_data = df_valid.pivot_table(index='education', columns='age', values='query_response', aggfunc='mean')

sns.heatmap(heatmap_data, cmap='plasma', annot=True, fmt='.1f', cbar_kws={'label': 'Query Response'})

## Tests

### Assumption Checks (for ANOVA and t-tests)

In [ ]:
check_normality_for_groups(df_valid, 'query_response', df_valid['age'])
check_variance_homogeneity(df_valid, 'query_response', df_valid['age'])

In [ ]:
check_normality_for_groups(df_valid, 'query_response', df_valid['gender'])
check_variance_homogeneity(df_valid, 'query_response', df_valid['gender'])

In [ ]:
check_normality_for_groups(df_valid, 'query_response', df_valid['education'])
check_variance_homogeneity(df_valid, 'query_response', df_valid['education'])

### Kruskal–Wallis Test

In [ ]:
kruskal_wallis_test(df_valid, 'query_response', 'age')

In [ ]:
kruskal_wallis_test(df_valid, 'query_response', 'gender')

In [ ]:
kruskal_wallis_test(df_valid, 'query_response', 'education')

## Dunn's Test

In [ ]:
dunns_test(df_valid, 'query_response', 'age')

In [ ]:
detailed_dunns_test(df_valid, 'query_response', 'age', total_comparisons=110)

In [ ]:
detailed_dunns_test(df_valid, 'query_response', 'gender', total_comparisons=110)

In [ ]:
detailed_dunns_test(df_valid, 'query_response', 'education', total_comparisons=110)

In [ ]:
age_edu_dunns = []
for i in df_valid['age'].unique():
    df_edu = df[df['age'] == i]
    curr_df = detailed_dunns_test(df_edu, 'query_response', 'education', total_comparisons=110)
    curr_df['age'] = i
    curr_df = curr_df.groupby(['age', 'education1', 'education2']).mean()
    age_edu_dunns.append(curr_df)
age_edu_dunns_results = pd.concat(age_edu_dunns)
age_edu_dunns_results['reject_p05'] = age_edu_dunns_results['reject_p05'].astype(bool)
age_edu_dunns_results['reject_p0005'] = age_edu_dunns_results['reject_p0005'].astype(bool)

age_edu_dunns_results

In [ ]:
print("Elapsed time:", datetime.datetime.now() - start_run)